# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
def establish_cols(sudoku):
    """
    Creates an array containing arrays of the squares in each 9 columns of the sudoku
    """
    cols=[[], [], [], [], [], [], [], [], []]
    for j in [0,1,2,3,4,5,6,7,8]:
        cols[j] = [[0,j], [1,j], [2,j], [3,j], [4,j], [5,j], [6,j], [7,j], [8,j]]
    return cols
    
    
def establish_rows(sudoku):
    """
    Creates an array containing arrays of the squares in each 9 rows of the sudoku
    """
    rows=[[], [], [], [], [], [], [], [], []]
    for i in [0,1,2,3,4,5,6,7,8]:
        rows[i] = [[i,0], [i,1], [i,2], [i,3], [i,4], [i,5], [i,6], [i,7], [i,8]]
    return rows


def establish_grid(sudoku):
    """
    Creates an array containing arrays of the squares in each of the 9 grids that make up the sudoku board
    
    Grid 1 is the left-most top-most grid and the function moves to the right until
    the right-most bottom-most grid is reached (grid 9)
    """
    grid1=[[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    
    grid2=[[0,3], [0,4], [0,5], [1,3], [1,4], [1,5], [2,3], [2,4], [2,5]]
    
    grid3=[[0,6], [0,7], [0,8], [1,6], [1,7], [1,8], [2,6], [2,7], [2,8]]
    
    grid4=[[3,0], [3,1], [3,2], [4,0], [4,1], [4,2], [5,0], [5,1], [5,2]]
    
    grid5=[[3,3], [3,4], [3,5], [4,3], [4,4], [4,5], [5,3], [5,4], [5,5]]
    
    grid6=[[3,6], [3,7], [3,8], [4,6], [4,7], [4,8], [5,6], [5,7], [5,8]]
    
    grid7=[[6,0], [6,1], [6,2], [7,0], [7,1], [7,2], [8,0], [8,1], [8,2]]
    
    grid8=[[6,3], [6,4], [6,5], [7,3], [7,4], [7,5], [8,3], [8,4], [8,5]]
    
    grid9=[[6,6], [6,7], [6,8], [7,6], [7,7], [7,8], [8,6], [8,7], [8,8]]
              
    grids = [grid1, grid2, grid3, grid4, grid5, grid6, grid7, grid8, grid9]
    
    return grids
    

    
def find_grid(i, j, sudoku):
    """
    Finds relevant grid containing given square [i,j]
    
    Input
        Address of square
        Sudoku
        
    Output
        1x9 array
            Returns all of the squares in the relevant grid containing the input square

    """  
    grids = establish_grid(sudoku)
    
    if 0<=i<=2 and 0<=j<=2:
        grid = grids[0]
        
    if 0<=i<=2 and 3<=j<=5:
        grid = grids[1]
            
    if 0<=i<=2 and 6<=j<=8:
        grid = grids[2]
              
    if 3<=i<=5 and 0<=j<=2:
        grid = grids[3]      

    if 3<=i<=5 and 3<=j<=5:
        grid = grids[4]  
       
    if 3<=i<=5 and 6<=j<=8:
        grid = grids[5] 
        
    if 6<=i<=8 and 0<=j<=2:
        grid = grids[6] 
        
    if 6<=i<=8 and 3<=j<=5:
        grid = grids[7]
        
    if 6<=i<=8 and 6<=j<=8:
        grid = grids[8] 
        
    return grid

def establish_neighbours(i,j,sudoku): 
    """
    Finds all squares in the same row, column and grid for a given square [i,j]  
    
    Input
        Address of square 
        Sudoku
        
    Output
        Array containing all neighbouring squares of the input square (with no duplicates and excluding the input square)
    
    """
    cols = establish_cols(sudoku)
    rows = establish_rows(sudoku)
    grid = find_grid(i,j,sudoku)
    row = rows[i]
    row.remove([i,j])        #removes input square from row
    col = cols[j]
    col.remove([i,j])        #removes input square from column
    new_grid = grid
    new_grid.remove([i,j])   #removes input square from grid
    neighbours = row + col 
   
    
    
    i=0            #removes all squares in grid that are already in the row/column
    while i < len(new_grid):
        if any(new_grid[i] == neighbours[j] for j in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]):
            new_grid.remove(new_grid[i])        
        else:
            i +=1
    
    neighbours = neighbours + new_grid         
        
    return neighbours  


In [3]:
import copy

def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """
    
    if is_valid(sudoku):                     #returns false if input sudoku is invalid
        backtrack_search(sudoku)
        if backtrack_search(sudoku) is False: #returns false if no solution exists
            return nullify(sudoku)
    else:
        return nullify(sudoku)
    
    solved_sudoku = sudoku 
    
    return solved_sudoku                              
          


def get_values(sudoku):
    """
    Establishes all possible values for each square given the assigned values of it's neighbours
    
    Input
        Sudoku
        
    Output
        Dictionary of length 81 where the keys are the addresses of the squares ([row, col])
        and the values are arrays of possible values for each square
    """
    values = {}
    rows = establish_rows(sudoku)
    cols = establish_cols(sudoku)
    for row in [0,1,2,3,4,5,6,7,8]:        
        for col in [0,1,2,3,4,5,6,7,8]:
            if not sudoku[row, col] == 0:      #if there is already a value in the square, then that is the only possible value
                value = sudoku[row, col]
                d = {(row,col):[value]}
                values.update(d)
            else:
                d = {(row,col): [1,2,3,4,5,6,7,8,9]}     #initially allows all numbers 1-9 as possible values
                values.update(d) 
                neighbours = establish_neighbours(row,col,sudoku)
                for square in neighbours:
                    for number in values[(row,col)]:          
                        array = values[(row,col)]
                        if sudoku[square[0], square[1]] == number and number in array:  #removes possible value from a square
                            array.remove(number)                                        #if it is already assigned in a neighbouring square
                            d1 = {(row,col):array}
                            values.update(d1)
                        
                             
    return values


def nullify(sudoku):
    """
    Sets all values of the sudoku to -1
    Used to nullify sudoku once other functions have established that there is no solution
    """
    for row in [0,1,2,3,4,5,6,7,8]:           
        for col in [0,1,2,3,4,5,6,7,8]:
            sudoku[row,col] = -1.

    return sudoku
 
    
def constraint_prop1(sudoku):
    """
    Assigns value to a square if there is only possible value and removes this value from neighbouring squares
    
    Input
        Sudoku
        
    Output
        Sudoku with newly-established values through constraint propagation
        Array holding all the squares that were edited. (This is used during backtracking.)
    """
    edited = []
    cols = establish_cols(sudoku)
    all_squares = cols[0] + cols[1]+cols[2]+cols[3]+cols[4]+cols[5]+cols[6]+cols[7]+cols[8]
         
    values = get_values(sudoku)
    while any(len(values[square[0], square[1]]) == 1 for square in all_squares if sudoku[square[0], square[1]] == 0): 
        for square in all_squares:
            if len(values[square[0], square[1]]) == 0:    #returns false if there are no possible values for a square
                return False
            if len(values[square[0], square[1]]) == 1 and sudoku[square[0],square[1]] == 0: #assigns value if there is only one possible and the square is empty
                array = values[square[0], square[1]]
                correct_value = array[0]
                sudoku = assign_value(correct_value, square[0], square[1], sudoku)
                edited.append(square)          #adds edited square to the array
                values = get_values(sudoku)    #re-establishes all possible values to take the new assignment into account        
                    
    
    return [sudoku, edited]
    
    
def constraint_prop2(sudoku):
    """
    Assigns number to a square if there is only one possible position for this number in the relevant row, column or grid

    Input
        Sudoku
        
    Output
        Sudoku with newly-established values through constraint propagation
        Array holding all the squares that were edited. (This is used during backtracking.)
    """  

    values = get_values(sudoku)
    cols = establish_cols(sudoku)
    rows = establish_rows(sudoku)
    grids = establish_grid(sudoku)
    edited = []

    for i in [0,1,2,3,4,5,6,7,8]:  #checks each grid
        grid = grids[i]        
        occurences = {1:[], 2:[], 3:[],4:[],5:[],6:[],7:[],8:[],9:[]}   
        for square in grid:
            for number in [1,2,3,4,5,6,7,8,9]:
                if number in values[square[0], square[1]]:
                    occurences[number].append(square)      #stores square in dictionary with the number as it's key if the number is in the possible values of the square
        for number in [1,2,3,4,5,6,7,8,9]:
            if len(occurences[number]) == 0:        #returns false if there is no possible place for a number within the grid
                return False
            elif len(occurences[number]) == 1:     #only one possible place for a number within the grid 
                array = occurences[number]
                square = array[0]
                if sudoku[square[0], square[1]] == 0:       
                    sudoku = assign_value(number, square[0], square[1], sudoku) #assigns value if square is empty
                    values = get_values(sudoku)
                    edited.append(square)        #adds edited square to array
               
                              
    for i in [0,1,2,3,4,5,6,7,8]: #checks each column
        col = cols[i]
        occurences = {1:[], 2:[], 3:[],4:[],5:[],6:[],7:[],8:[],9:[]} 
        for square in col:
            for number in [1,2,3,4,5,6,7,8,9]:
                if number in values[square[0], square[1]]:
                    occurences[number].append(square)
        
        for number in [1,2,3,4,5,6,7,8,9]:
            if len(occurences[number]) == 0:
                return False
            elif len(occurences[number]) == 1:
                array = occurences[number]
                square = array[0]
                if sudoku[square[0], square[1]] == 0:
                    sudoku = assign_value(number, square[0], square[1], sudoku)
                    values = get_values(sudoku)
                    edited.append(square)
                
    
    for i in [0,1,2,3,4,5,6,7,8]: #checks each row
        row = rows[i]
        occurences = {1:[], 2:[], 3:[],4:[],5:[],6:[],7:[],8:[],9:[]} 
        for square in row:
            for number in [1,2,3,4,5,6,7,8,9]:
                if number in values[square[0], square[1]]:
                    occurences[number].append(square)
        for number in [1,2,3,4,5,6,7,8,9]:
            if len(occurences[number]) == 0:
                return False
            elif len(occurences[number]) == 1:
                array = occurences[number]
                square = array[0]
                if sudoku[square[0], square[1]] == 0:
                    sudoku = assign_value(number, square[0], square[1], sudoku)
                    values = get_values(sudoku)
                    edited.append(square)
                
    
    return [sudoku, edited]
    

    
def assign_value(value, i, j, sudoku):
    """
    Assigns given value to square [i,j] in the sudoku and returns the sudoku
    """
    sudoku[i,j] = value
    
    return sudoku
  
    
    
def empty_square(sudoku): 
    """
    Returns true if there is one or more empty squares in the sudoku and false if the sudoku is full
    """
    cols = establish_cols(sudoku)
    all_squares = cols[0] + cols[1]+cols[2]+cols[3]+cols[4]+cols[5]+cols[6]+cols[7]+cols[8] 
    
    for square in all_squares:
        if sudoku[square[0], square[1]] == 0:
            return True 
    return False 
        
    
    
def least_value(values, sudoku):  
    """
    Finds square in the sudoku with the least possible values (minimum remaining values heuristic)
    
    Input
        Values dictionary of the input sudoku
        Sudoku
        
    Output
        Square with least possible values
    """
    cols = establish_cols(sudoku)
    all_squares = cols[0] + cols[1]+cols[2]+cols[3]+cols[4]+cols[5]+cols[6]+cols[7]+cols[8] 
    
    if empty_square(sudoku):   #requires there to be at least one empty square
        for square in all_squares:
            if len(values[square[0], square[1]]) == 0:    #returns square if there are no possible values for this square
                return square
        if all(len(values[square[0], square[1]]) >= 1 for square in all_squares): 
            if all(len(values[square[0], square[1]]) == 1 for square in all_squares): #if all squares have only one value, then sudoku is solved
                min_length = 1 
            else:        #returns minimum length of possible values for a length greater than one
                min_length = min(len(values[square[0], square[1]]) for square in all_squares if len(values[square[0], square[1]]) > 1) 
            
            for square in all_squares:  #locates a square with minumum value length and returns it
                if len(values[square[0], square[1]]) == min_length and sudoku[square[0], square[1]] == 0:
                    return square

    
    
def remove_value(row, col, sudoku):
    """
    Removes the value from square [i,j] by assigning 0 to the square
    """
    sudoku = assign_value(0, row, col, sudoku)
    return sudoku                          


def is_valid(sudoku): 
    """
    Returns true if the given sudoku has no repeated non-zero values within a neighbourhood and all
    squares have at least one possible value, and false otherwise
    """
    cols = establish_cols(sudoku)
    all_squares = cols[0] + cols[1]+cols[2]+cols[3]+cols[4]+cols[5]+cols[6]+cols[7]+cols[8] 
    
    for item in all_squares:       #checks that there are no non-zero repeats within neighbourhoods
        neighbours = establish_neighbours(item[0], item[1], sudoku)
        for square in neighbours:
            if sudoku[square[0], square[1]] == sudoku[item[0], item[1]] and not sudoku[square[0], square[1]] == 0:
                return False
       
    
    values = get_values(sudoku)     #checks there is still at least one possible value for all sudoku squares
    if any(len(values[square[0], square[1]]) == 0 for square in all_squares):
        return False     
    
    return True


def is_solved(sudoku):
    """
    Returns true if the sudoku is solved and false otherwise
    """
    if not empty_square(sudoku) and is_valid(sudoku): #checks there are no empty squares and that the sudoku is valid
        return True     
    return False

In [4]:
def is_valid_move(value, row, col, sudoku):
    """
    Returns true if assigning value to square [row,col] would give a valid sudoku, and false otherwise
    
    Input
        Value 
        Row,Col that gives location of square to be tested
        Sudoku
        
    Output
         True or False
    """
    if not sudoku[row, col] == 0:   #returns false if square is not empty 
        return False
    values = get_values(sudoku)
    if value not in values[row,col]:    #returns false if value is not in possible values for given square
        return False
    else: 
        sudoku_copy = copy.deepcopy(sudoku)    #creates a deepcopy to test validity of the move 
        sudoku_copy = assign_value(value, row, col, sudoku_copy)
        if joint_constraint_prop(sudoku_copy) is False:    #uses both methods of constraint propagation to check validity of move
            return False
        else:
            return True
        
    

def joint_constraint_prop(sudoku):    
    """
    Combines both constraint propagation functions in order to solve the sudoku more efficiently
    
    Input
        Sudoku
        
    Output
        Sudoku with updated values established through constraint propagation
        Array holding all the squares that were edited. (This is used during backtracking)
    """    
    edited = [] 
    
    for i in range(1, 3):         #repeats both constraint propagation functions twice
        if is_solved(sudoku):
            return [sudoku, edited]      #terminates function if sudoku is solved 
        else:
            results = constraint_prop1(sudoku)
            if results is False:       
                return False
            else:
                sudoku = results[0] 
                array = results[1]
                for element in array:
                    edited.append(element)      #adds all edited squares to array
                results2 = constraint_prop2(sudoku)
                if results2 is False:
                    return False
                else:
                    sudoku = results2[0]
                    array1 = results2[1]
                    for element in array1:     #adds all edited squares to array
                        edited.append(element)
     
    
    return [sudoku, edited]


def backtrack_search(sudoku):
    """
    Implements a backtrack search, combined with constraint propagation and a minimum remaing values heuristic to 
    efficiently solve the given sudoku
    
    Input
        Sudoku
        
    Output
        True if solution found/exists and false otherwise
    """
    empty = empty_square(sudoku)
    
    if not empty:     #if there are no empty squares left, the sudoku is solved                
        return True  
    else:
        values = get_values(sudoku)
        least_square = least_value(values, sudoku)      
        for value in values[least_square[0], least_square[1]]:        #iterates through possible values for least_square
            if is_valid_move(value, least_square[0], least_square[1], sudoku):
                sudoku = assign_value(value, least_square[0], least_square[1], sudoku) 
                sudoku_copy = copy.deepcopy(sudoku)
                if joint_constraint_prop(sudoku_copy) is False:  #moves on to next value if sudoku is invalid after constraint propagation
                    pass
                else:
                    result = joint_constraint_prop(sudoku)
                    sudoku = result[0]
                    edited = result[1]
                    if backtrack_search(sudoku):
                        return True
                    for square in edited:                        #removes values from all the squares that were edited during the last constraint propagation
                        sudoku = remove_value(square[0], square[1], sudoku)
                sudoku = remove_value(least_square[0], least_square[1], sudoku)         
        #triggers backtracking
        return False        #returns false if all of the possible values for least_square lead to an invalid sudoku and forces a backtrack 
        

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [5]:
SKIP_TESTS = True

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [6]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [7]:
# This is a TEST CELL. Do not delete or change.